In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
from os.path import realpath, normpath

'C:\\Users\\Muttabi Hudaya\\AppData\\Local\\Programs\\Python\\Python38\\Lib\\site-packages\\cv2\\python-3.8\\cv2.cp38-win_amd64.pyd'

In [3]:
cv2.cuda.getCudaEnabledDeviceCount()

1

In [151]:
PATH_TO_ORB_TEST_IMG = 'results/cropped/e-ktp/'
img_ = PATH_TO_ORB_TEST_IMG + '10_card_crop_0.jpg'
img2 = PATH_TO_ORB_TEST_IMG + '7_card_crop_0.jpg'

In [157]:
# Check smaller width dimension of 2 images
# Smaller images: check width x dimension and choose the smaller
# Resize higher image resolution with the smaller dimension (width / height)
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is Noner
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized

def get_downscaled_images(img1, img2):
    img1_h, img1_w, = img1.shape
    img2_h, img2_w, = img2.shape
    
    if img1_h > img2_h:
        scaled_img1 = image_resize(img1, width=img2_w)
        scaled_img2 = img2
    else:
        scaled_img2 = image_resize(img2, width=img1_w)
        scaled_img1 = img1
        
    
    return scaled_img1, scaled_img2

In [ ]:
print('Original Dimensions : ',img.shape)
 
width = 350
height = 450
dim = (width, height)
 
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

In [ ]:
f, (ax1, ax2) = plt.subplots(1,2)

img_show1 = cv2.imread(img_)
img_show1 = cv2.cvtColor(img_show1, cv2.COLOR_BGR2RGB)
img_show2 = cv2.imread(img2)
img_show2 = cv2.cvtColor(img_show2, cv2.COLOR_BGR2RGB)

plt.rcParams['figure.figsize'] = [20,10]
ax1.set_title('Image 1')
ax1.imshow(img_show1)
ax2.set_title('Image 2')
ax2.imshow(img_show2)
plt.show()

In [7]:
img_show1.shape

(471, 690, 3)

In [8]:
img_show2.shape

(342, 550, 3)

In [9]:
rescaled_img1, rescaled_img2 = get_downscaled_images(img_show1, img_show2)

In [10]:
rescaled_img1.shape

(375, 550, 3)

In [11]:
rescaled_img2.shape

(342, 550, 3)

In [ ]:
# Load images

f, (ax1, ax2) = plt.subplots(1,2)
plt.rcParams['figure.figsize'] = [20,10]
ax1.set_title('Colored image')
ax1.imshow(rescaled_img1)
ax2.set_title('Grayed image')
ax2.imshow(rescaled_img2, cmap='gray')
plt.show()

In [ ]:
# Load images
img1 = cv2.imread(img_)
img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)

f, (ax1, ax2) = plt.subplots(1,2)
plt.rcParams['figure.figsize'] = [20,10]
ax1.set_title('Colored image')
ax1.imshow(img1)
ax2.set_title('Grayed image')
ax2.imshow(img1_gray, cmap='gray')
plt.show()

In [147]:
img1=rescaled_img1
img1_gray=cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
img2=rescaled_img2

In [56]:
# Get ORB Keypoints and Descriptor
n_features = 3000
factor = 2.0
orb = cv2.ORB_create(n_features)
corb = cv2.cuda_ORB.create(n_features)

def get_orb_kp_and_des(img_path, with_cuda=False):
    img1 = cv2.imread(img_path)
    if with_cuda:
        cuMat1 = cv2.cuda_GpuMat()
        cuMat1.upload(img1)
        cuMat1_gray = cv2.cuda.cvtColor(cuMat1, cv2.COLOR_BGR2GRAY)
        print("here")
        _kps1, _descs1 = corb.detectAndComputeAsync(cuMat1_gray, None)
        keypoints_img = corb.convert(_kps1)
        descriptor_img = _descs1
    else:
        img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        keypoints_img, descriptor_img =  orb.detectAndCompute(img1_gray, None)
    return keypoints_img, descriptor_img

In [ ]:
# Extracts key points
n_features = 3000
factor = 2.0
orb = cv2.ORB_create(n_features, factor)

PATH_TO_ORB_TEST_IMG = 'results/cropped/e-ktp/'
img_ = PATH_TO_ORB_TEST_IMG + '10_card_crop_0.jpg'
img2 = PATH_TO_ORB_TEST_IMG + '7_card_crop_0.jpg'
npMat1 = cv2.imread(img_)
npMat2 = cv2.imread(img2)
# upload into Cuda
cuMat1 = cv2.cuda_GpuMat()
cuMat2 = cv2.cuda_GpuMat()
cuMat1.upload(npMat1)
cuMat2.upload(npMat2)        
#convert to Gray
cuMat1g = cv2.cuda.cvtColor(cuMat1, cv2.COLOR_BGR2GRAY)
cuMat2g = cv2.cuda.cvtColor(cuMat2, cv2.COLOR_BGR2GRAY)

#ORB
corb = cv2.cuda_ORB.create()
_kps1, _descs1 = corb.detectAndComputeAsync(cuMat1g, None)
_kps2, _descs2 = corb.detectAndComputeAsync(cuMat2g, None)

#convert Keypoints to CPU
kps1 = corb.convert(_kps1)
kps2 = corb.convert(_kps2) 

#BruteForce Matching
cbf = cv2.cuda_DescriptorMatcher.createBFMatcher(cv2.NORM_HAMMING)
cmatches = cbf.match(_descs1, _descs2) 
# Sort matches by score
cmatches.sort(key=lambda x: x.distance, reverse=False)

print(len(cmatches))

# Remove not so good matches
# numGoodMatches = int(len(cmatches) * 0.15)
# cmatches = cmatches[:numGoodMatches]  
# Draw top matches
imMatches = cv2.drawMatches(npMat1, kps1, npMat2, kps2, cmatches[:73], None)
plt.title('Best Matching Points')
plt.imshow(imMatches)
plt.show()


f, (ax1, ax2) = plt.subplots(1,2)

img_show1 = cv2.imread(img_)
img_show1 = cv2.cvtColor(img_show1, cv2.COLOR_BGR2RGB)
img_show2 = cv2.imread(img2)
img_show2 = cv2.cvtColor(img_show2, cv2.COLOR_BGR2RGB)

plt.rcParams['figure.figsize'] = [20,10]
ax1.set_title('Image 1')
ax1.imshow(img_show1)
ax2.set_title('Image 2')
ax2.imshow(img_show2)
plt.show()

In [176]:
sift = cv2.SIFT_create()

In [177]:
# Feature Matching
img_to_match = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

keypoints_img, descriptor_img = sift.detectAndCompute(img1_gray, None) # No Mask
keypoints_img_to_match, descriptor_img_to_match = sift.detectAndCompute(img_to_match, None) # No Mask

bf = cv2.BFMatcher()

matches_keypoints_img_to_match = bf.knnMatch(descriptor_img, descriptor_img_to_match, k=2)

In [161]:
# Feature Matching
PATH_TO_ORB_TEST_IMG = 'results/cropped/e-ktp/'
img_ = PATH_TO_ORB_TEST_IMG + '10_card_crop_0.jpg'
img2 = PATH_TO_ORB_TEST_IMG + '7_card_crop_0.jpg'
img2 = cv2.imread(img2)
img1_gray = cv2.cvtColor(cv2.imread(img_), cv2.COLOR_BGR2GRAY)
img_to_match = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
img1_gray, img_to_match = get_downscaled_images(img1_gray, img_to_match)

keypoints_img, descriptor_img = orb.detectAndCompute(img1_gray, None) # No Mask
keypoints_img_to_match, descriptor_img_to_match = orb.detectAndCompute(img_to_match, None) # No Mask

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# matches_keypoints_img_to_match = bf.knnMatch(descriptor_img, descriptor_img_to_match, k=1)
# Match descriptors.
matches_keypoints_img_to_match = bf.match(descriptor_img,descriptor_img_to_match)
# Sort them in the order of their distance.
matches_keypoints_img_to_match = sorted(matches_keypoints_img_to_match, key = lambda x:x.distance)


numGoodMatches = int(len(matches_keypoints_img_to_match) * 0.15)

print(len(keypoints_img))
print(len(keypoints_img_to_match))
print(len(matches_keypoints_img_to_match))

2917
2917
837


In [162]:
# Apply ratio test
good = []
for m in matches_keypoints_img_to_match:
    if m.distance < 64:
        good.append(m)
        
print(len(keypoints_img))
print(len(matches_keypoints_img_to_match))
print(len(good))

2917
837
789


In [ ]:
# Draw first 10 matches.
img3 = cv2.drawMatches(img1_gray,keypoints_img,img_to_match,keypoints_img_to_match,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.imshow(img3),plt.show()

In [178]:
# Apply ratio test
good = []
bad = []
for m,n in matches_keypoints_img_to_match:
    if m.distance < 0.75*n.distance:
        good.append([m])
    else:
        bad.append([m])
        
print(len(matches_keypoints_img_to_match))
print(len(good))

# husen to husen : 31
# husen to suhu (card) : 24
# suhu to suhu (blurred): 17

971
130


In [179]:
print(len(keypoints_img_to_match))
print(len(keypoints_img))
print(len(good))

391
971
130


In [ ]:
result = cv2.drawMatchesKnn(img1_gray,keypoints_img, img_to_match, keypoints_img_to_match,good ,None,flags=2)
plt.title('Best Matching Points')
plt.imshow(result)
plt.show()